<a href="https://colab.research.google.com/github/liorber13/BI-Project-3rd-year-2nd-semester/blob/main/STTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install jsonpath_ng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import os
import csv
import json
import numpy as np 
import pandas as pd 
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline 

In [58]:
data = []
with open("Loan_data_for_BI_csv.csv", "r") as csvFile:
    csvReader = csv.DictReader(csvFile)
    for item in csvReader:
        data.append(item)
data[:3]

[{'\ufeffID': '10266',
  'Gender': 'Female',
  'Married': 'Yes',
  'Age': '50',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Applicant_Income': '47090',
  'Coapplicant_Income': '0',
  'Load_Date': 'מאי-22',
  'Loan_Amount': '1130000',
  'Term': '360',
  'Credit_History': '1',
  'Area': 'Semiurban'},
 {'\ufeffID': '10311',
  'Gender': 'Male',
  'Married': 'Yes',
  'Age': '48',
  'Dependents': '3+',
  'Education': 'Not Graduate',
  'Self_Employed': 'Yes',
  'Applicant_Income': '40090',
  'Coapplicant_Income': '17770',
  'Load_Date': 'ספט-22',
  'Loan_Amount': '1130000',
  'Term': '360',
  'Credit_History': '1',
  'Area': 'Urban'},
 {'\ufeffID': '10716',
  'Gender': 'Male',
  'Married': 'Yes',
  'Age': '59',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Applicant_Income': '26000',
  'Coapplicant_Income': '7000',
  'Load_Date': 'נוב-22',
  'Loan_Amount': '960000',
  'Term': '360',
  'Credit_History': '1',
  'Area': 'Semiurban'}]

# New Section

In [59]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given field name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """


In [60]:
class TransformMask(Enum):
    # add here any masks you want
    CLEAN_STRING = ".strip().lower()"
    CAPITAL_LETTER = ".strip().lower().title()"
    NUM_DATA = ".strip()"

In [61]:
class Database():
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }

    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.strip().get("id") == self.id:
                return x
        return None

    def add_source(self, id, source_field_name, source_field_mapping, source_field_type, is_required):
        self.db["source"].append(
            {
                "id": id,
                "source_field_name": source_field_name,
                "source_field_mapping": source_field_mapping,
                "source_field_type": source_field_type,  # use python types
                "is_required": is_required,
            })

    def add_destination(self, id, destination_field_name, destination_field_mapping, destination_field_type, default_value):
        self.db["destination"].append(
            {
                "id": id,
                "destination_field_name": destination_field_name,
                "destination_field_mapping": destination_field_mapping,
                "destination_field_type": destination_field_type,  # use python types
                "default_value": default_value,
            })

    def add_transform(self, id, transform_mask):
        self.db["transform"].append(
            {
                "id": id,
                "transform_mask": transform_mask
            })

    def add_mapping(self, id, mapping_source, mapping_destination, mapping_transform):
        self.db["mapping"].append(
            {
                "id": id,
                "mapping_source": mapping_source,
                "mapping_destination": mapping_destination,
                "mapping_transform": mapping_transform
            })

    @property
    def get_data_source_target_mapping(self):
        return self.db

In [62]:
New_data = Database()

In [63]:
New_data.add_source(1, "ID", "ID", int, True)
New_data.add_source(2, "Gender", "Gender", str, True)
New_data.add_source(3, "Married", "Married", str, True)
New_data.add_source(4, "Age", "Age", float, True)
New_data.add_source(5, "Dependents", "Dependents", int, True)
New_data.add_source(6, "Education", "Education", str, True)
New_data.add_source(7, "Self_Employed", "Self_Employed", str, True)
New_data.add_source(8, "Applicant_Income", "Applicant_Income", float, True)
New_data.add_source(9, "Coapplicant_Income", "Coapplicant_Income", float, True)
New_data.add_source(10, "Load_Date", "Load_Date", str, True)
New_data.add_source(11, "Loan_Amount", "Loan_Amount", float, True)
New_data.add_source(12, "Term", "Term", int, True)
New_data.add_source(13, "Credit_History", "Credit_History", int, True)
New_data.add_source(14, "Area", "Area", str, True)

New_data.add_destination(1, "ID", "ID", int, "-1")
New_data.add_destination(2, "Gender", "Gender", str, "n/a")
New_data.add_destination(3, "Married", "Married", str, "n/a")
New_data.add_destination(4, "Age", "Age", float, "-1")
New_data.add_destination(5, "Dependents", "Dependents", int, "-1")
New_data.add_destination(6, "Education", "Education", str, "n/a")
New_data.add_destination(7, "Self_Employed", "Self_Employed", str, "n/a")
New_data.add_destination(8, "Applicant_Income", "Applicant_Income", float, "-1")
New_data.add_destination(9, "Coapplicant_Income", "Coapplicant_Income", float, "-1")
New_data.add_destination(10, "Load_Date", "Load_Date", str, "n/a")
New_data.add_destination(11, "Loan_Amount", "Loan_Amount", float, "-1")
New_data.add_destination(12, "Term", "Term", int, "-1")
New_data.add_destination(13, "Credit_History", "Credit_History", int, "n/a")
New_data.add_destination(14, "Area", "Area", str, "n/a")

New_data.add_transform(1, 'CLEAN_STRING')
New_data.add_transform(2, 'CAPITAL_LETTER')
New_data.add_transform(3, 'NUM_DATA')

New_data.add_mapping(1, 1, 1, 3)
New_data.add_mapping(2, 2, 2, 1)
New_data.add_mapping(3, 3, 3, 1)
New_data.add_mapping(4, 4, 4, 3)
New_data.add_mapping(5, 5, 5, 3)
New_data.add_mapping(6, 6, 6, 2)
New_data.add_mapping(7, 7, 7, 1)
New_data.add_mapping(8, 8, 8, 3)
New_data.add_mapping(9, 9, 9, 3)
New_data.add_mapping(10, 10, 10, 1)
New_data.add_mapping(11, 11, 11, 3)
New_data.add_mapping(12, 12, 12, 3)
New_data.add_mapping(13, 13, 13, 3)
New_data.add_mapping(14, 14, 14, 2)

In [64]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)
        # Database.get_data_by_field(self)
        # Database.get_data_by_id(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")
    
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None
class Target(Interface, Database):
    def __init__(self):
        Database.__init__(self)
        # Database.get_data_by_field(self)
        # Database.get_data_by_id(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)
        # Database.get_data_by_field(self)
        # Database.get_data_by_id(self)

    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)
        # Database.get_data_by_field(self)
        # Database.get_data_by_id(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

In [73]:
class STTM():
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):
        for mappings in self._get_mapping_data():
            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [74]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)


{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{

In [46]:
pd.DataFrame(transformed_data)

pd.DataFrame(transformed_data).groupby("Area").mean()

KeyError: ignored